In [2]:
import pandas as pd

#그래프에 한글 표현하기
import  matplotlib.pyplot as plt
import  matplotlib.font_manager as fm



#마이너스 표시하기
import  matplotlib as mlp
mlp.rcParams["axes.unicode_minus"]  = False

import numpy as np
import pandas  as pd
import seaborn as sns
##회귀분석
import statsmodels.api as sm
# 정규화
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import  RFE
from lightgbm import LGBMClassifier,  LGBMRegressor
from lightgbm import plot_importance, plot_metric, plot_tree
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR, SVC
import multiprocessing 
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LinearRegression
from scipy import stats
#사용법이 파이선과 다른 기존xgb를  파이선에 맞게 만들어 놓은 것
from  xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import RobustScaler

#변수영향도 그래프, 트리그래프 더 쉽게 그리게 해줌
from xgboost import plot_importance, plot_tree

from sklearn.metrics import mean_absolute_error, mean_squared_error

import shap

# 1. 절도폭력

In [8]:
df = pd.read_csv("../1_최종데이터셋/1_절도폭력안전도.csv")
df = df[["crm_clue_고소","crm_wthr_바람","crm_clue_현행범","crm_clue_피해자신고","vic_sx_2","vic_age_60세초과","cctv_개수",
         "배치인원_수","비상벨_개수",
        "일인가구수","기초수급_19세이하","외국인수", "score_절폭"]]

# train, test 분리
X_train, X_test, y_train, y_test  = train_test_split(df.iloc[:,:-1], df["score_절폭"],test_size = 0.3, random_state=0)

#train
ridge = Ridge(alpha=10)
ridge.fit(X_train, y_train)
mean_absolute_error(ridge.predict(X_test), y_test)

#20년도 테스트셋
df_test = pd.read_csv("../2_20년도예측데이터(테스트셋)/v2_1_절도폭력_20년도test.csv")

# 예측값
ridge.predict(df_test.iloc[:,2:-1])

array([79.33783178, 76.03806677, 77.08884472, 75.43285277, 74.74765844,
       78.01137997, 75.81963518, 74.83206881, 77.45791819, 85.61356479,
       79.26991829, 76.25482523, 78.27752627, 74.75926062, 80.47664254,
       79.15626031, 77.78449914, 79.95733931, 80.63718405, 77.31661662,
       80.34171708, 77.578371  , 81.83828787, 81.6555933 , 77.72234686,
       79.59936847, 83.69781285, 79.6008245 , 80.11286424, 81.23873892,
       85.01502915, 80.84323001, 78.49992938, 81.15227493, 78.26626324,
       81.39996873, 76.66684515, 75.28411261, 79.01164627, 78.56051791,
       79.8563127 ])

# 2. 강도살인

In [10]:
df = pd.read_csv("../1_최종데이터셋/2_강도살인안전도.csv")
df = df[["crm_clue_변사체","crm_clue_자수","crm_clue_진정","crm_clue_현행범","crm_tm_저녁"
    ,"vic_sx_1","강도살인_검거수","화재_사망","화재_부상","공원_개수","자살_사망률_10만명당"
         ,"자살_연령표준화사망률_10만명당","기초수급_35_59세","기초수급_60세이상","외국인수", "score_강살"]]

#20년도 테스트셋
df_test = pd.read_csv("../2_20년도예측데이터(테스트셋)/v2_2_강도살인_20년도test.csv").drop(columns = "화재_부동산피해_천원",axis = 1)
df_test.head()

# 전체스케일링을 위해 df,테스트셋 병합
df_con = pd.concat([df, df_test.iloc[:,2:]], axis = 0).reset_index(drop=True)


#각 변수들을 스케일링 하는 함수
def robust_scaling(df, scale_columns):
    for col in scale_columns:
        series_med = df[col].median()
        series_per75 = np.percentile(df[col],75)
        series_per25 = np.percentile(df[col],25)
        df[col] = df[col].apply(lambda x: (x-series_med)/(series_per75-series_per25))      
    return df

#변사체는 0,1,2밖에없어서 스케일링 안되므로 제외하고 함
robust_scaling(df_con, df_con.iloc[:,3:-1].columns)

# train, test 분리
X_train, X_test, y_train, y_test  = train_test_split(df_con.iloc[:123,:-1], df_con.iloc[:123,-1],test_size = 0.3, random_state=0)

#SVR
svr = SVR(kernel="linear")
svr.fit(X_train, y_train)

#결과
svr.predict(df_con.iloc[123:,:-1])

array([78.90082716, 74.80310855, 79.91166523, 79.14424625, 78.68539344,
       80.25482811, 78.78097893, 78.56928676, 81.34719684, 79.6609002 ,
       78.95868152, 78.23575285, 72.38185064, 77.15254348, 79.18235948,
       79.15784745, 78.12351082, 82.95041739, 81.50090422, 80.86822561,
       76.45498249, 80.13397991, 81.68279646, 81.90733088, 78.23525741,
       79.82141417, 83.13902399, 80.48497014, 81.19391407, 80.87880785,
       82.52979213, 79.76373683, 75.88059904, 82.2039901 , 78.99552166,
       82.06346429, 81.2703204 , 77.75736155, 82.25770542, 78.89658902,
       81.36126876])

# 3. 교통사고

In [11]:
df = pd.read_csv("../1_최종데이터셋/3_교통사고안전도.csv")
df = df[["crm_wthr_눈","crm_clue_피해자신고","vic_age_2_30대","일인가구수"
    ,"총_인구수","기초수급_19세이하","기초수급_60세이상"
         ,"외국인수", "score_교통"]]

# train, test 분리
X_train, X_test, y_train, y_test  = train_test_split(df.iloc[:,:-1], df["score_교통"],test_size = 0.3, random_state=0)

elastic = ElasticNet(alpha=1)
elastic.fit(X_train, y_train)

#20년도 테스트셋
df_test = pd.read_csv("../2_20년도예측데이터(테스트셋)/v2_3_교통사고_20년도test.csv", index_col=0)

# 결과값
elastic.predict(df_test.iloc[:,2:-1])

array([70.51241012, 70.87405828, 70.73839358, 69.01550306, 69.51174618,
       68.82297373, 69.39002214, 69.39364011, 69.44608647, 68.9148595 ,
       72.09815712, 71.03385396, 69.22284364, 66.31714551, 69.05838132,
       70.59315781, 70.42406637, 71.67066763, 73.01628225, 70.30188592,
       70.28264435, 69.70244   , 69.39877671, 71.34822931, 69.6566278 ,
       70.77094593, 71.26720968, 70.70293426, 69.99196516, 71.48411872,
       73.12897135, 71.61852976, 69.78429614, 70.40330109, 70.96883645,
       69.30682339, 70.83349788, 70.76046932, 70.97309953, 68.95444283,
       69.45225639])

# 4. 법질서

In [4]:
df = pd.read_csv("../1_최종데이터셋/4_법질서안전도.csv")
df = df[["crm_clue_자수","crm_clue_타인신고","crm_clue_현행범","crm_clue_탐문정보","vic_sx_2","기초수급_19세이하","기초수급_20_34세",
         "crm_tm_새벽","화재_사망","cctv_개수","배치인원_수","비상벨_개수","일인가구수", "score_법질서"]]

#20년도 테스트셋
df_test = pd.read_csv("../2_20년도예측데이터(테스트셋)/v2_4_법질서_20년도test.csv", index_col=0)

# df, test셋 데이터 병합
df_con = pd.concat([df, df_test.iloc[:,2:]], axis = 0).reset_index(drop=True)

#각 변수들을 스케일링 하는 함수
def minmax_scaling(df, scale_columns):
    for col in scale_columns:
        series_min = df[col].min()
        series_max = df[col].max()
        df[col] = df[col].apply(lambda x: (x-series_min)/(series_max-series_min))
    return df

# min_max 스케일링
minmax_scaling(df_con, df_con.columns[:-1])

# train, test 분리
X_train, X_test, y_train, y_test  = train_test_split(df_con.iloc[:123,:-1], df_con.iloc[:123,-1],test_size = 0.3, random_state=0)


ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
#결과
ridge.predict(df_con.iloc[123:,:-1])

array([75.07841226, 72.09154659, 72.76550054, 71.89781998, 71.95019563,
       70.13060636, 71.28172785, 72.30584252, 73.55254145, 75.2763887 ,
       73.60873606, 73.18214612, 72.37616279, 69.94141363, 73.15346172,
       70.9608711 , 69.6200891 , 72.30531664, 78.33181267, 71.91320463,
       71.22204834, 72.58020814, 72.81124687, 74.39606277, 71.66113946,
       74.25452622, 75.56985159, 73.17460628, 73.78484841, 73.7374928 ,
       75.77821277, 74.11375188, 69.39493776, 71.15946574, 71.95991787,
       71.65287065, 71.69062262, 69.40185503, 72.01314567, 70.21693829,
       71.51923397])

# 5. 전반적

In [5]:
df = pd.read_csv("../1_최종데이터셋/5_전반적안전도.csv")
df = df[["crm_wthr_흐림","crm_clue_타인신고","crm_clue_피해자신고","crm_clue_현행범","crm_tm_저녁","화재_사망","배치인원_수",
        "비상벨_개수","기초수급_19세이하","외국인수", "score_전반"]]

#20년도 데이터
df_test = pd.read_csv("../2_20년도예측데이터(테스트셋)/v2_5_전반_20년도test.csv", index_col=0)

# train, test 분리
X_train, X_test, y_train, y_test  = train_test_split(df.iloc[:,:-1], df["score_전반"],test_size = 0.3, random_state=0)

ridge = Ridge(alpha=100)
ridge.fit(X_train, y_train)
# 결과값
ridge.predict(df_test.iloc[:,2:-1])

array([75.95989961, 72.85882024, 74.12546092, 72.88238337, 73.25244934,
       73.83028126, 72.96391303, 71.95103459, 74.79693073, 79.02541529,
       74.61045898, 73.0150204 , 73.38261547, 73.97775225, 74.45724932,
       77.36973924, 74.27047262, 77.39792781, 78.25811691, 74.48358414,
       75.73781919, 75.38579454, 77.11554208, 77.23379399, 74.9037694 ,
       75.32728136, 80.97643681, 75.88300791, 76.23399524, 76.69777454,
       77.76699856, 76.86577092, 77.88419152, 76.39934239, 74.18951032,
       77.06665681, 74.79907464, 72.78584711, 76.33380096, 76.33714363,
       75.79545103])

# 중간점수 체크(결과제출양식으로 만들어주기)

In [8]:
#절도폭력
절폭_list = [79.33783178, 76.03806677, 77.08884472, 75.43285277, 74.74765844,
       78.01137997, 75.81963518, 74.83206881, 77.45791819, 85.61356479,
       79.26991829, 76.25482523, 78.27752627, 74.75926062, 80.47664254,
       79.15626031, 77.78449914, 79.95733931, 80.63718405, 77.31661662,
       80.34171708, 77.578371  , 81.83828787, 81.6555933 , 77.72234686,
       79.59936847, 83.69781285, 79.6008245 , 80.11286424, 81.23873892,
       85.01502915, 80.84323001, 78.49992938, 81.15227493, 78.26626324,
       81.39996873, 76.66684515, 75.28411261, 79.01164627, 78.56051791,
       79.8563127 ]

#강도살인
강살_list =[78.90082716, 74.80310855, 79.91166523, 79.14424625, 78.68539344,
       80.25482811, 78.78097893, 78.56928676, 81.34719684, 79.6609002 ,
       78.95868152, 78.23575285, 72.38185064, 77.15254348, 79.18235948,
       79.15784745, 78.12351082, 82.95041739, 81.50090422, 80.86822561,
       76.45498249, 80.13397991, 81.68279646, 81.90733088, 78.23525741,
       79.82141417, 83.13902399, 80.48497014, 81.19391407, 80.87880785,
       82.52979213, 79.76373683, 75.88059904, 82.2039901 , 78.99552166,
       82.06346429, 81.2703204 , 77.75736155, 82.25770542, 78.89658902,
       81.36126876]

#교통사고
교통_list =[70.51241012, 70.87405828, 70.73839358, 69.01550306, 69.51174618,
       68.82297373, 69.39002214, 69.39364011, 69.44608647, 68.9148595 ,
       72.09815712, 71.03385396, 69.22284364, 66.31714551, 69.05838132,
       70.59315781, 70.42406637, 71.67066763, 73.01628225, 70.30188592,
       70.28264435, 69.70244   , 69.39877671, 71.34822931, 69.6566278 ,
       70.77094593, 71.26720968, 70.70293426, 69.99196516, 71.48411872,
       73.12897135, 71.61852976, 69.78429614, 70.40330109, 70.96883645,
       69.30682339, 70.83349788, 70.76046932, 70.97309953, 68.95444283,
       69.45225639]

#법질서
법질서_list = [75.07841226, 72.09154659, 72.76550054, 71.89781998, 71.95019563,
       70.13060636, 71.28172785, 72.30584252, 73.55254145, 75.2763887 ,
       73.60873606, 73.18214612, 72.37616279, 69.94141363, 73.15346172,
       70.9608711 , 69.6200891 , 72.30531664, 78.33181267, 71.91320463,
       71.22204834, 72.58020814, 72.81124687, 74.39606277, 71.66113946,
       74.25452622, 75.56985159, 73.17460628, 73.78484841, 73.7374928 ,
       75.77821277, 74.11375188, 69.39493776, 71.15946574, 71.95991787,
       71.65287065, 71.69062262, 69.40185503, 72.01314567, 70.21693829,
       71.51923397]

#전반
전반_list = [75.95989961, 72.85882024, 74.12546092, 72.88238337, 73.25244934,
       73.83028126, 72.96391303, 71.95103459, 74.79693073, 79.02541529,
       74.61045898, 73.0150204 , 73.38261547, 73.97775225, 74.45724932,
       77.36973924, 74.27047262, 77.39792781, 78.25811691, 74.48358414,
       75.73781919, 75.38579454, 77.11554208, 77.23379399, 74.9037694 ,
       75.32728136, 80.97643681, 75.88300791, 76.23399524, 76.69777454,
       77.76699856, 76.86577092, 77.88419152, 76.39934239, 74.18951032,
       77.06665681, 74.79907464, 72.78584711, 76.33380096, 76.33714363,
       75.79545103]

# 결과제출 양식으로 맞춰주기

In [10]:
df_station = pd.read_csv("../1_최종데이터셋/5_전반적안전도.csv").iloc[:41,:1]
df_result = pd.DataFrame({"절도폭력":절폭_list,"강도살인":강살_list,"교통사고":교통_list,"법질서":법질서_list,"전반적":전반_list})
df_final = pd.concat([df_station,df_result], axis = 1)

#범죄안전도,  분야별 안전도,  종합체감안전도  점수  계산
df_final["범죄안전도"] = (df_final["절도폭력"]+df_final["강도살인"])/2
df_final["분야별안전도"] = df_final["범죄안전도"]*0.343+df_final["교통사고"]*0.305+df_final["법질서"]*0.352
df_final["종합체감안전도_점수"] = df_final["전반적"]*0.3+df_final["분야별안전도"]*0.7
df_final = df_final[["관할서","종합체감안전도_점수","분야별안전도","범죄안전도","절도폭력","강도살인","교통사고","법질서","전반적"]]\


df_final["관할서"] = df_final["관할서"].str.replace("경기","")
df_final["관할서"] = df_final["관할서"].str.replace("경남","")
df_final["관할서"] = df_final["관할서"].str.replace("충남","")

df_final = df_final.rename(columns = {"관할서":"경찰서명","종합체감안전도_점수":"종합체감안전도_점수   \n(전반적안전도*0.3) + (분야별안전도*0.7)",
                          "분야별안전도":"분야별 안전도\n(범죄안전도*0.343) + \n(교통사고안전도*0.305)\n + (법질서준수도*0.352)",
                          "범죄안전도":"범죄 안전도\n(절도폭력안전도+강도살인안전도)/2",
                          "절도폭력":"문항1. 절도 폭력안전도\n (문항1 점수 * 10)",
                          "강도살인":"문항2. 강도 살인 안전도\n(문항2 점수 * 10)",
                          "교통사고":"문항3. 교통사고 안전도\n(문항3 점수*10)",
                          "법질서":"문항4. 법질서 준수도\n(문항 4점수 * 10)",
                          "전반적":"문항5. 전반적 안전도\n(문항 5점수 * 10)"})

df_양식 = pd.read_csv("../../raw_data/25.결과제출양식.csv")
df_양식["경찰서명"] =df_양식["경찰서명"].str.strip()
df_ap = df_양식
df_ap.head()

# 반복문으로 양식대로 기입
for i in range(len(df_ap)):
    for z in range(len(df_final)):
        if df_ap["경찰서명"][i] ==df_final["경찰서명"][z]:
            df_ap.iloc[i,2:] = df_final.iloc[z,1:]
            
        else:
            pass
            
df_ap


df_ap["경찰서명"]=df_양식["경찰서명"]
df_ap

,순번,경찰서명,종합체감안전도_점수 \n(전반적안전도*0.3) + (분야별안전도*0.7),분야별 안전도\n(범죄안전도*0.343) + \n(교통사고안전도*0.305)\n + (법질서준수도*0.352),범죄 안전도\n(절도폭력안전도+강도살인안전도)/2,문항1. 절도 폭력안전도\n (문항1 점수 * 10),문항2. 강도 살인 안전도\n(문항2 점수 * 10),문항3. 교통사고 안전도\n(문항3 점수*10),문항4. 법질서 준수도\n(문항 4점수 * 10),문항5. 전반적 안전도\n(문항 5점수 * 10)
0,1,서울중부경찰서,75.157322,74.653116,80.634676,79.011646,82.257705,70.973100,72.013146,76.333801
1,2,서울종로경찰서,75.196056,74.394370,81.731717,81.399969,82.063464,69.306823,71.652871,77.066657
2,3,서울남대문경찰서,75.894172,75.249705,81.453878,79.957339,82.950417,71.670668,72.305317,77.397928
3,4,서울서대문경찰서,73.722836,73.216722,77.978802,77.722347,78.235257,69.656628,71.661139,74.903769
4,5,서울혜화경찰서,73.827096,72.751361,78.728553,78.560518,78.896589,68.954443,70.216938,76.337144
5,6,서울용산경찰서,75.095519,74.536738,81.678133,81.152275,82.203990,70.403301,71.159466,76.399342
6,7,서울성북경찰서,75.358949,74.983928,80.653389,80.112864,81.193914,69.991965,73.784848,76.233995
7,8,서울동대문경찰서,74.099247,73.397002,78.398350,80.341717,76.454982,70.282644,71.222048,75.737819
8,9,서울마포경찰서,75.522699,74.840052,81.760542,81.838288,81.682796,69.398777,72.811247,77.115542
9,10,서울영등포경찰서,73.896500,72.187489,77.190264,78.499929,75.880599,69.784296,69.394938,77.884192


# 결과제출양식 형식으로 저장

In [48]:
#df_ap.to_csv("../최종결과/v53_최종결과.csv", index = 0)